<center>
    <h1> Nearest Neighbor </h1>
</center>

<hr>

### Imports

In [1]:
import pandas as pd
import numpy  as np
import fasttext

from dataset     import shuffle_and_get_dataset, es_significativo
from normalizer  import preprocesar_codigo, combinar_y_normalizar
from utilitarios import get_enunciado

In [2]:
def preprocess_to_txt(data, fname, normalizer):
    """
    Preprocesa los datos y los guarda en un txt en un formato usable para FastText
    para el entrenamiento no supervisado
    Args:
        data        :  Samples del dataset 
        fname       :  Nombre del archivo resultante
        normalizer  :  Funcion de normalizacion de strings
    """
    with open(fname, 'w') as file:
        for i in range(len(data)):
            code = normalizer(data[i])
            file.write(f"{code}\n")

### Obtencion de datos

In [3]:
# Lectura de datos
discussions_df = pd.read_csv("./Data/discussions_anon.csv")
items_df       = pd.read_csv("./Data/items_anon.csv")

In [4]:
# Filas del archivo items que corresponden al lenguaje JavaScript
JS_ID = 5   # Identificador de codigo JavaScript dentro de la base de datos

item_js    = items_df[items_df["language_id"] == JS_ID]
item_id_js = item_js["id"]

# Filas del archivo discussion que corresponden al lenguaje JavaScript
discussion_js = discussions_df[discussions_df.item_id.isin(item_id_js)]

# Datasets 
data   = discussion_js[["solution", 
                        "test_results",
                        "expectation_results", 
                        "item_id",
                        "id"]].values.astype(str)

target = discussion_js["submission_status"].values.astype(str)


# Hay dos etiquetas que no parecen significativas para la tarea: aborted y pending
# por lo que las omitire de los datos
mask = [sample not in ["aborted", "pending"] for sample in target]
data   = data[mask]
target = target[mask]

# Obtenemos los datasets que usaremos para entrenar los modelos
(train_data, train_target), (val_data, val_target), \
(test_data, test_target) = shuffle_and_get_dataset(data, target, 
                                                   val_prop=0.0, 
                                                   test_prop=0.2)

len(train_data), len(val_data), len(test_data), len(np.unique(target))

(25056, 0, 6264, 4)

### Filtrado de datos no significados

In [5]:
def filtrar_no_significativo(data, target):
    """
    Filtra las muestras no significativas y sus respectivas labels. Las muestras
    del dataset se asumen que en su primera coordenada tienen codigo
    Args:
        data      :  Muestras del dataset
        target    :  label para cada muestra
    """
    # Filtramos los datos que no son informativos con un boolmap
    boolmap = []
    for code, label in zip(data[:, 0], target):
        boolmap.append(es_significativo(code, label))
        
    # Datos no informativos filtrados
    data   = data[boolmap]
    target = target[boolmap]
    
    return data, target

In [6]:
f_train_data, f_train_target  = filtrar_no_significativo(train_data, train_target)
f_val_data  , f_val_target    = filtrar_no_significativo(val_data  , val_target)
f_test_data , f_test_target   = filtrar_no_significativo(test_data , test_target)

len(f_train_data), len(f_val_data), len(f_test_data)

(24691, 0, 6171)

### Documento de entrenamiento

In [7]:
# Creamos el archivo de entrenamiento de FastText
dataset_dir = "./FastText_Datasets/" # Directorio de datasets
preprocess_to_txt( f_train_data[:, 0:3], 
                   dataset_dir + "clustering.txt", 
                   combinar_y_normalizar )

### Definicion del Word Embedding

In [8]:
dimension = 30 
wng       = 3
epocas    = 15

word_embedding = fasttext.train_unsupervised(
    dataset_dir + "clustering.txt", # Dataset de entrenamiento 
    model = "skipgram",             # Tipo de modelo
    dim=dimension,                  # Dimension del word embedding
    wordNgrams=wng,                 # Tamaño del ngram
    epoch=epocas,                   # Cantidad de epocas
    ws=5,                           # Tamaño del context window
    loss="ns",                      # Negative sampling
    verbose=0
)

# <code, test_results, expectation_results> -> Vector
# Usaremos estos vectores para asociar a una nueva muestra (las del conjunto de test)
# codigos similares
train_vectors = np.array([word_embedding.get_sentence_vector(combinar_y_normalizar(code)) for \
                          code in f_train_data[:, 0:3]])

In [9]:
# Distancia euclidea entre dos puntos
euclidean = lambda x, y: np.linalg.norm(x-y) 

def k_nearest_neighbors(k, sample, vectores):
    """
    Retorna el indice de los k vectores mas cercanos al vector
    del word embedding asociado a sample
    Args:
        k           :    Cantidad de indices a retornar 
        sample      :    Una muestra del conjunto de datos
        vectores    :    Vectores de 'entrenamiento' producidos por el 
                         word embedding
    """
    codigo_preprocesado = combinar_y_normalizar(sample)
    codigo_vectorizado  = word_embedding.get_sentence_vector(codigo_preprocesado)

    distancias = np.array([euclidean(codigo_vectorizado, v) for \
                           v in vectores])
        
    k_nn = distancias.argsort()[:k] # Indices de los K vectores mas cercanos 
    
    return k_nn


def print_k_nn(k, sample, id_enunciado, vectores):
    """
    Imprime el codigo dado junto a su enunciado y a los 
    k codigos mas cercanos. Los codigos se imprimen sin normalizar, 
    por lo que aparecen tal cual como se escribieron. 
    Args:
        k             :  Cantidad de codigos similares
        sample        :  Muestra del conjunto de datos (<code, test_results, expectation_results>)
        id_enunciado  :  Enunciado asociado a la muestra sample
        vectores      :  Vectores de 'entrenamiento' producidos por el 
                         word embedding
    """
    indices = k_nearest_neighbors(k, sample, vectores)
    
    print(get_enunciado(int(id_enunciado), item_js)) # Imprime el enunciado
    print("CODIGO DADO\n")
    print(sample[0])                                 # Imprime el codigo dado
    print(f"CODIGOS CERCANOS\n")   
    for i in indices:                                # Imprime los codigos cercanos
        print(f_train_data[i][0])
        print(f"\n {'#'*80} \n")

### Scripts de prueba

El siguiente script toma una muestra aleatoria del conjunto de datos no vistos por el modelo, busca los k codigos mas cercanos y los imprime por pantalla

In [10]:
desired_label = "failed"    # Para elegir una muestra aleatoria con ese submission_status

# Eleccion de una muestra aleatoria con la label deseada
choice = np.random.randint(len(f_test_data[f_test_target == desired_label]))
muestra_elegida = f_test_data[f_test_target == desired_label][choice]

print_k_nn(
    k = 3,                                  # Cantidad de codigos cercanos a imprimir   
    sample       = muestra_elegida[:3],     # <codigo, test_result, expectation_result>
    id_enunciado = muestra_elegida[3],      # item_id de la muestra
    vectores     = train_vectors            # vectores calculados por el word embedding
)

[Operando strings - 2769] :: ¿Y qué podemos hacer con los strings, además de compararlos? ¡Varias cosas! Por ejemplo, podemos preguntarles cuál es su cantidad de letras:

```javascript
ム longitud("biblioteca")
10
ム longitud("babel")
5
```

O también podemos _concatenarlos_, es decir, obtener **uno nuevo** que junta dos strings:

```javascript
ム "aa" + "bb"
"aabb"
ム "sus anaqueles " + "registran todas las combinaciones"
"sus anaqueles registran todas las combinaciones"
```

O podemos preguntarles si uno comienza con otro:

```javascript
ム comienzaCon("una página", "una")
true
ム comienzaCon("la biblioteca", "todos los fuegos")
false
```

> Veamos si queda claro: definí la función `longitudNombreCompleto`, que tome un nombre, un segundo nombre y un apellido, y retorne su longitud total, contando dos espacios extra para separarlos:
>
>```javascript
> ム longitudNombreCompleto("Cosme", "Miguel", "Fulanito")
>21
>```
CODIGO DADO

function longitudNombreCompleto (Nombre, Apellido) {
  return l

### Otro script de prueba

Este es completamente analogo al anterior pero ahora imprime la discussion_id de los ejercicios cercanos

In [11]:
desired_label = "failed"    # Para elegir una muestra aleatoria con ese submission_status

# Eleccion de una muestra aleatoria con la label deseada
choice = np.random.randint(len(f_test_data[f_test_target == desired_label]))
muestra_elegida = f_test_data[f_test_target == desired_label][choice]

indices = k_nearest_neighbors (
    k = 3,                                  # Cantidad de codigos cercanos a imprimir  
    sample       = muestra_elegida[:3],     # <codigo, test_result, expectation_result>
    vectores     = train_vectors            # vectores calculados por el word embedding
)

print(f"Id seleccionada: {muestra_elegida[4]}")
print(f"Ids de codigos cercanos:")
for i in indices:
    print(f_train_data[i][4])

Id seleccionada: 239
Ids de codigos cercanos:
7928
28834
14211
